In [2]:
# VGG网络, 首先实现VGG Block.
import torch
from torch import nn
from d2l import torch as d2l


In [1]:
# 首先实现VGG块: 
def vgg_block(num_Conv2ds, in_channels, out_channels):
    layers = []
    for _ in range(num_Conv2ds):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    # 最后添加一个MaxPooling(), 实现宽高减半. (卷积层已经
    # 实现了通道数翻倍)
    layers.append(nn.MaxPool2d(kernel_size=3, stride=2))


def get_vgg_block(num_Conv12ds, in_channels, out_channels):
    layers = [nn.Conv2d(in_channels, out_channels, 3, 1, 1)]
    
    for _ in range(num_Conv12ds-1):
        layers.append(nn.Conv2d(out_channels, out_channels, 3, 1, 1))
        layers.append(nn.ReLU())

    # 最后实现宽高减半
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [ ]:
# 提前高度你的架构:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
# 这是非常经典的结构. 
# 其中每一个元素